In [ ]:
"""
it will read all text file from given folder,
and create folder for each file and save, word cloud, word counting, keyword in files,
"""

In [168]:
files_location = '/content/drive/MyDrive/text_tech_classification'
min_counts_for_keywords = 2 # use words that appear minimum 2 times in docoment.
key_word_score = 0.99 # 90%

In [141]:
import string
from collections import Counter
import re
import nltk
# downlaoding nltk stop words
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from nltk.probability import FreqDist
import os, glob
from nltk.corpus import words
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [142]:
def paragraph_process(data):
  data = data.split(';')
  all_data = []
  for i in data:
    all_data.extend(i.split('\t'))
  data = []
  for i in all_data:
    data.extend(i.split('.'))
  all_data = []
  for i in data:
    if not len(i.strip().split()) <= 1:
      all_data.append(i.lower())
  return all_data

def file_process(data):
  all_data = []
  for i in data:
    all_data.extend(paragraph_process(i))
  return all_data 

# function to clean stop words
def cleaning_stopwords(text, words_dict = set(words.words()),
                       STOPWORDS = set(stopwords.words('english'))):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS if word in  words_dict if len(word)>2])
# Function to clean punctuation
def cleaning_punctuations(text, punctuations_list = string.punctuation):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
# Funvtion to clean repreating characters
def cleaning_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
# Funtion to removing email and mentions signs
def cleaning_email(data):
    return re.sub('@[^\s]+', ' ', data)
# Cleaning web urls
def cleaning_URLs(data):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',data)
# Cleaning numebrs 
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
# Function to stemming all words
def stemming_on_text(data, st = nltk.PorterStemmer()):
    text = [st.stem(word) for word in data]
    return text
# Function to lemmatization all words
def lemmatizer_on_text(data, lm = nltk.WordNetLemmatizer()):
    text = [lm.lemmatize(word) for word in data]
    return text
# Funtion to apply all cleaning function at once
def preprocesses_data(data, tokenizer = RegexpTokenizer(r'\w+')):
  x = cleaning_stopwords(data)
  x = cleaning_punctuations(x)
  x = cleaning_email(x)
  x = cleaning_URLs(x)
  x = cleaning_numbers(x)
  x =  tokenizer.tokenize(x)
  # We are not doing stemming here because we need to make word thier base word by understading context of the data.
  # That's why we choose lammatization
  
  # data= data.apply(lambda x: stemming_on_text(x))
  x = lemmatizer_on_text(x)
  # data = data.apply(lambda x: check_words(x))
  x = ' '.join(x)
  # print(x)
  return x

In [146]:
def process_file(file_name):
    print(file_name)
    with open(file_name, "r") as txt_file:
        data = txt_file.readlines()
    data = file_process(data)
    tqdm.pandas(desc = 'Cleaning data')
    data = pd.Series(data).progress_apply(lambda x: preprocesses_data(x))

    data = list(filter(None, data))
    data = " ".join(data)

    fdist = FreqDist(data.split())
    most_common = fdist.most_common(20)
    print(most_common)
    most_common = list(reversed(most_common))

    counts = np.array(most_common)[:,1]
    words = np.array(most_common)[:,0]

    plt.figure(figsize = (15,7))
    plt.bar(list(words), list(counts))
    plt.xticks(rotation='90')
    plt.title('Most common words')
    plt.show()
    plt.savefig('Most common words.png')

    # Generate a word cloud image
    wordcloud = WordCloud().generate(data)
    plt.figure(figsize = (15,7))
    plt.title('words Word cloud')
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.show()
    plt.savefig('words Word cloud.png')

    word2vec = Word2Vec([data.split()], min_count=min_counts_for_keywords)

    all_words = []
    for word in tqdm(data.split(),desc='Finding keywords'):
      try:
        sim_words = word2vec.wv.most_similar(word)
        for sim_word in sim_words:
          if sim_word[1]>key_word_score:
            all_words.append(word+' '+ sim_word[0])
      except:
        pass
    fdist = FreqDist(all_words)
    most_common = fdist.most_common(20)
    print(most_common)
    most_common = list(reversed(most_common))

    counts = np.array(most_common)[:,1]
    words = np.array(most_common)[:,0]

    plt.figure(figsize = (15,7))
    plt.bar(list(words), list(counts))
    plt.xticks(rotation='90')
    plt.title('Most common keywords')
    plt.show()
    plt.savefig('Most common keywords.png')
    
    new_all_words = list(set(all_words))
    counts = dict(Counter(all_words))
    # open file in write mode
    with open(r'keywords.txt', 'w') as fp:
        for item in counts:
          try:
            item = item +'\t' +str(counts[item])
          except:
            item = item +'\t' +str(1)
          # write each item on a new line
          fp.write("%s\n" % item)
        print('Done')

/content/drive/MyDrive/text_tech_classification/storage capacity_2001_3000.txt


Cleaning data: 100%|██████████| 39529/39529 [00:01<00:00, 19855.44it/s]

[('storage', 5434), ('capacity', 2837), ('data', 2402), ('corp', 1957), ('application', 1427), ('method', 1403), ('system', 1371), ('memory', 1258), ('device', 1150), ('patent', 1023), ('power', 1006), ('description', 1000), ('use', 934), ('based', 921), ('energy', 896), ('advantage', 860), ('novelty', 770), ('unit', 735), ('information', 735), ('electric', 678)]


In [153]:
files = []
file_ = os.path.join(files_location,'*.txt')
files.extend(glob.glob(file_))

for file_name in files:
  folder_path = file_name[:-4]
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)
  os.chdir(folder_path)
  print(folder_path)
  process_file(file_name)

/content/drive/MyDrive/text_tech_classification/storage capacity_2001_3000
/content/drive/MyDrive/text_tech_classification/storage capacity_1_1000
/content/drive/MyDrive/text_tech_classification/data transfer performance 
/content/drive/MyDrive/text_tech_classification/storage capacity_4001_5000
/content/drive/MyDrive/text_tech_classification/storage capacity_1001_2000
/content/drive/MyDrive/text_tech_classification/savedrecs
/content/drive/MyDrive/text_tech_classification/storage capacity_3001_4000
